In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# 시가총액 상위 300개 기업리스트 (미래에셋 대우 API 미확인으로, 2021.09.12 기준 시총 상위 300개 기업 사용)
lists = pd.read_csv("/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv")
lists_1 = lists.loc[0:299]
lists_2 = lists_1[['종목명']]


# ESG 키워드
ESG = {
'E_keyword' : ['친환경', '탄소', '에너지'],
'S_keyword' : ['캠페인', '교육', '사회'],
'G_keyword' : ['위원회', '이사', '지배구조']}
ESG_keyword = pd.DataFrame.from_dict(ESG)

print(ESG_keyword)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  E_keyword S_keyword G_keyword
0       친환경       캠페인       위원회
1        탄소        교육        이사
2       에너지        사회      지배구조


In [ ]:
!pip install selenium
!pip install beautifulsoup4
!pip install tqdm

     |████████████████████████████████| 904 kB 5.3 MB/s 


In [ ]:
# 지난 N개월 간의 종목명 + E 관련 뉴스기사 본문 추출
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import urllib.request
from urllib.parse import quote
import pandas as pd


# 크롤링 메인 함수
def get_news(stock, keyword, query, sort, lastpage, period):

  stockName = stock
  ESGKeyword = keyword

  # 크롤링 결과 저장 데이터프레임
  news_df = pd.DataFrame(columns=("stockName", "ESGKeyword", "Title", "Article")) 

  idx = 0


  for month in range (0, period): # 해당 검색어(query)로 N(=period)개월치 기사 추출

    s_date = datetime.date.today() + timedelta(days = -30 * (month+1))
    e_date = datetime.date.today() + timedelta(days = -30 * month)

    pageNum = 1

    for i in range (1, int(lastpage)*10, 10):
      print(f"/{pageNum}페이지입니다/")

      url_query = quote(query)
      url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={url_query}&sort=0&photo=0&field=0&pd=6&ds={s_date}&de={e_date}&nso=so:r,p:{period}m,a:all&start={i}"

      search_url = urllib.request.urlopen(url).read()
      soup = BeautifulSoup(search_url, 'html.parser')
      links = soup.find_all('div', {'class':'info_group'})
  

      # 하나의 검색 페이지 안에서의 반복
      for link in links:

        # '네이버 뉴스' 형식으로 작성된 기사만을 추출 (href 2번)
        news_urls = link.find_all('a')

        if (len(news_urls) == 1): # '네이버 뉴스' 형식 X
          continue

        else:  # '네이버 뉴스' 형식 O
          
          for news_url_test in news_urls:
            news_url = news_url_test.get('href')

          news_link = urllib.request.urlopen(news_url).read()
          news_html = BeautifulSoup(news_link, 'html.parser')

          title = news_html.find('h3', {'id': 'articleTitle'}).get_text()
          article = news_html.find('div', {'id': 'articleBodyContents'}).get_text()
          article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
          article = article.replace("function _flash_removeCallback() {}", "")
          article = article.replace("\n","")
          article = article.replace("\t","")

          news_df.loc[idx] = [stockName, ESGKeyword, title, article]
          idx += 1
          print("#", end="")


      pageNum += 1


  return news_df




In [ ]:
# E(환경)부문


# CONCAT 목적의 초기 데이터프레임 형성
default = {
'stockName' : ['Test'],
'ESGKeyword' : ['Test'],
'Title' : ['Test'],
'Article' : ['Test']}
df_default = pd.DataFrame.from_dict(default) 


# 파라미터 입력 후 크롤링
for i in range(300): # 300: 시총 상위 300개 기업
  
  for j in range(3): # ESG 키워드 개수 총 3개

    query = lists_2['종목명'].values[i] + "+" + ESG_keyword['E_keyword'].values[j]
    sort = "0" # 관련도 순 기사
    lastpage = 100 # 100 페이지까지 크롤링 (최대 400페이지까지 가능하기는 하나, 파일 크기 과부하 방지)
    period = 6 # 6개월간의 기사

    print(lists_2['종목명'].values[i])
    print(ESG_keyword['E_keyword'].values[j])
    
    df = get_news(lists_2['종목명'].values[i], ESG_keyword['E_keyword'].values[j], query, sort, lastpage, period)
    df_default = pd.concat([df_default, df], axis = 0)


df_default.head()
df_default.to_csv('E부문.csv', header = True, index = False)

삼성전자
친환경
/1페이지입니다/
########/2페이지입니다/
######/3페이지입니다/
#######/4페이지입니다/
#######/5페이지입니다/
####/6페이지입니다/
######/7페이지입니다/
####/8페이지입니다/
#/9페이지입니다/
####/10페이지입니다/
####/11페이지입니다/
###/12페이지입니다/
#####/13페이지입니다/
#####/14페이지입니다/
###/15페이지입니다/
###/16페이지입니다/
######/17페이지입니다/
####/18페이지입니다/
#/19페이지입니다/
##/20페이지입니다/
##/21페이지입니다/
#/22페이지입니다/
#/23페이지입니다/
######/24페이지입니다/
###/25페이지입니다/
###/26페이지입니다/
###/27페이지입니다/
#####/28페이지입니다/
###/29페이지입니다/
######/30페이지입니다/
#####/31페이지입니다/
###/32페이지입니다/
###/33페이지입니다/
##/34페이지입니다/
##/35페이지입니다/
##/36페이지입니다/
##/37페이지입니다/
##/38페이지입니다/
##/39페이지입니다/
###/40페이지입니다/
###/41페이지입니다/
##/42페이지입니다/
###/43페이지입니다/
##/44페이지입니다/
#####/45페이지입니다/
####/46페이지입니다/
#####/47페이지입니다/
####/48페이지입니다/
####/49페이지입니다/
####/50페이지입니다/
####/51페이지입니다/
#####/52페이지입니다/
#####/53페이지입니다/
###/54페이지입니다/
##/55페이지입니다/
##/56페이지입니다/
##/57페이지입니다/
#/58페이지입니다/
######/59페이지입니다/
#######/60페이지입니다/
######/61페이지입니다/
####/62페이지입니다/
##/63페이지입니다/
##/64페이지입니다/
##/65페이지입니다/
###/66페이지입니다/
###/67페이지입니다/
###/68페이지입니다/
####/69페이지입니

KeyboardInterrupt: ignored

In [ ]:
# S(사회)부문


# CONCAT 목적의 초기 데이터프레임 형성
default = {
'stockName' : ['Test'],
'ESGKeyword' : ['Test'],
'Title' : ['Test'],
'Article' : ['Test']}
df_default = pd.DataFrame.from_dict(default) 


# 파라미터 입력 후 크롤링
for i in range(300): # 300: 시총 상위 300개 기업
  
  for j in range(3): # ESG 키워드 개수 총 3개

    query = lists_2['종목명'].values[i] + "+" + ESG_keyword['S_keyword'].values[j]
    sort = "0" # 관련도 순 기사
    lastpage = 100 # 100 페이지까지 크롤링 (최대 400페이지까지 가능하기는 하나, 파일 크기 과부하 방지)
    period = 6 # 6개월간의 기사

    print(lists_2['종목명'].values[i])
    print(ESG_keyword['S_keyword'].values[j])
    
    df = get_news(lists_2['종목명'].values[i], ESG_keyword['S_keyword'].values[j], query, sort, lastpage, period)
    df_default = pd.concat([df_default, df], axis = 0)


df_default.head()
df_default.to_csv('S부문.csv', header = True, index = False)

국내 기업에 대해 **G**를 평가하기에는 
G에 대한 전반적인 국내 기업의 실천 정도가 미흡하기 때문에**큰 의미가 있지 않다고 판단**해, **G 키워드는 배제**

또한 크롤링해야 하는 양이 너무 많아 코스피 시총 탑 100으로 추리고, E 관련 키워드만 조사하기로 함